In [4]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [5]:
# Set the file path
filepath = Path("../Resources/fb_data.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

,Date,Open,High,Low,Close,Volume,Volatility,sentiment,RSI
0,2019-01-02,128.99,137.51,128.56,135.4100,25465924,0.286522,-0.025997,NaN
1,2019-01-03,134.69,137.17,131.12,131.8902,18011900,0.088181,-0.057317,NaN
2,2019-01-04,134.01,138.00,133.75,137.9400,23927539,0.164677,0.083206,NaN
3,2019-01-07,137.45,138.87,135.91,138.2300,16261533,0.021965,-0.016295,NaN
4,2019-01-08,139.89,143.14,139.54,142.5843,22239823,0.104831,0.039262,NaN


In [8]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = aapl_df.loc[:, ["Date", "Close"]].copy()

# Set the short window and long windows
short_window = 5
long_window = 15

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2019-01-02,135.4100,NaN,NaN,0.0,NaN
2019-01-03,131.8902,NaN,NaN,0.0,0.0
2019-01-04,137.9400,NaN,NaN,0.0,0.0
2019-01-07,138.2300,NaN,NaN,0.0,0.0
2019-01-08,142.5843,137.21090,NaN,0.0,0.0
2019-01-09,144.2500,138.97890,NaN,0.0,0.0
2019-01-10,144.1500,141.43086,NaN,0.0,0.0
2019-01-11,143.7205,142.58696,NaN,0.0,0.0
2019-01-14,145.6300,144.06696,NaN,0.0,0.0


In [9]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)